# 📉 Backtest Trading Strategy – Crypto ML
Generate buy/sell signals using ML predictions and evaluate strategy performance.

## 📦 Step 1: Load data and predictions

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load dataset again
df = pd.read_csv("btc_usd.csv", parse_dates=["Date"])
df.set_index("Date", inplace=True)

# Recreate lag features
df["lag_1"] = df["Close"].shift(1)
df["lag_2"] = df["Close"].shift(2)
df["lag_3"] = df["Close"].shift(3)
df["target"] = df["Close"].shift(-1)
df.dropna(inplace=True)

# Re-train simple XGBoost model for this notebook
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

X = df[["lag_1", "lag_2", "lag_3"]]
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

model = XGBRegressor()
model.fit(X_train, y_train)

df_test = df.iloc[-len(X_test):].copy()
df_test["pred"] = model.predict(X_test)


## 📈 Step 2: Define signal generation logic

In [ ]:

# Simple signal logic: if tomorrow's predicted price > today's close → BUY
df_test["signal"] = np.where(df_test["pred"] > df_test["Close"], 1, -1)
df_test["daily_return"] = df_test["Close"].pct_change().fillna(0)
df_test["strategy_return"] = df_test["signal"].shift(1) * df_test["daily_return"]


## 📊 Step 3: Visualize performance

In [ ]:

(1 + df_test["strategy_return"]).cumprod().plot(label="Strategy", figsize=(12,5))
(1 + df_test["daily_return"]).cumprod().plot(label="Market")
plt.title("Cumulative Returns – Strategy vs Market")
plt.legend()
plt.grid(True)
plt.show()


🚀 Backtesting complete! You can now optimize and evaluate more complex strategies.